### Imports

In [2]:
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier



### Desafio:

Este é o Trabalho 1 de avaliação da disciplina EEL891 (Introdução ao Aprendizado de Máquina) para a turma do período 2020-1.

Neste trabalho você construirá um classificador para apoio à decisão de aprovação de crédito.

A ideia é identificar, dentre os clientes que solicitam um produto de crédito (como um cartão de crédito ou um empréstimo pessoal, por exemplo) e que cumprem os pré-requisitos essenciais para a aprovação do crédito, aqueles que apresentem alto risco de não conseguirem honrar o pagamento, tornando-se inadimplentes.

Para isso, você receberá um arquivo com dados históricos de 20.000 solicitações de produtos de créditos que foram aprovadas pela instituição, acompanhadas do respectivo desfecho, ou seja, acompanhadas da indicação de quais desses solicitantes conseguiram honrar os pagamentos e quais ficaram inadimplentes.

Com base nesses dados históricos, você deverá construir um classificador que, a partir dos dados de uma nova solicitação de crédito, tente predizer se este solicitante será um bom ou mau pagador.

O objetivo da competição é disputar com seus colegas quem consegue obter a acurácia mais alta em um conjunto de 5.000 solicitações de crédito aprovadas (diferentes das 20.000 anteriores) cujos desfechos (quitação da dívida ou inadimplência) são mantidos ocultos no site do Kaggle, que medirá automaticamente a taxa de acerto das previsões enviadas pelos competidores, sem revelar o "gabarito".

### Leitura de dados

In [58]:
df_from_eda = pd.read_csv('data/desafio 1/dataset_from_eda_2021-05-02.csv')
print(df_from_eda.columns)

test = df_from_eda[df_from_eda['origem'] == 'teste']
train = df_from_eda[df_from_eda['origem'] == 'treino']


print(test.shape)
print(train.shape)
# print('\n')
# print(f'test columns {test.columns}')
# print('\n')
# print(f'train columns {train.columns}')
# print('\n')
# print(list(set(train.columns.tolist()) - set(test.columns.tolist())))

Index(['id_solicitante', 'produto_solicitado', 'dia_vencimento', 'sexo',
       'idade', 'estado_civil', 'possui_telefone_residencial',
       'meses_na_residencia', 'valor_patrimonio_pessoal_norm', 'profissao',
       'ocupacao', 'inadimplente', 'idade_bin', 'renda_total_norm',
       'estado_onde_nasceu', 'estado_onde_reside', 'regiao_onde_reside',
       'regiao_onde_nasceu', 'origem'],
      dtype='object')
(5000, 19)
(20000, 19)


In [6]:
# test = pd.read_csv('data/desafio 1/conjunto_de_teste.csv')
# train = pd.read_csv('data/desafio 1/conjunto_de_treinamento.csv')

# print(test.shape)
# print(train.shape)
# print('\n')
# print(f'test columns {test.columns}')
# print('\n')
# print(f'train columns {train.columns}')
# print('\n')
# print(list(set(train.columns.tolist()) - set(test.columns.tolist())))

In [9]:
test.isna().sum() 

id_solicitante                      0
produto_solicitado                  0
dia_vencimento                      0
sexo                                0
idade                               0
estado_civil                        0
possui_telefone_residencial         0
meses_na_residencia               362
valor_patrimonio_pessoal_norm       0
profissao                         762
ocupacao                          690
inadimplente                     5000
idade_bin                           0
renda_total_norm                    0
estado_onde_nasceu                  0
estado_onde_reside                  0
regiao_onde_reside                  0
regiao_onde_nasceu                  0
origem                              0
dtype: int64

### Data Cleaning

In [59]:
# Transform text columns
def encode_string_col(df, col):
    df[col] = df[col].astype('category').cat.codes
    
# sexo 
df_from_eda['sexo'].fillna('N')
encode_string_col(df_from_eda, 'sexo')

# estado_onde_nasceu ... etc (general string columns)
string_cols = ['estado_onde_nasceu', 'estado_onde_reside', 'possui_telefone_residencial',
               'regiao_onde_reside', 'regiao_onde_nasceu']

for col in string_cols:
    encode_string_col(df_from_eda, col)

# train test split 

train = df_from_eda[df_from_eda['origem'] == 'treino']
test = df_from_eda[df_from_eda['origem'] == 'teste']

# dropar variável alvo do teste e origem

test.drop(columns=['inadimplente', 'origem'], inplace=True)
train.drop(columns='origem', inplace=True)

# fill nan

test.fillna(0, inplace=True)
train.fillna(0, inplace=True)

print(test.shape)
print(train.shape)


(5000, 17)
(20000, 18)


/home/hilsts/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/home/hilsts/.local/lib/python3.8/site-packages/pandas/core/frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


### Cleaned test

In [60]:
test

,id_solicitante,produto_solicitado,dia_vencimento,sexo,idade,estado_civil,possui_telefone_residencial,meses_na_residencia,valor_patrimonio_pessoal_norm,profissao,ocupacao,idade_bin,renda_total_norm,estado_onde_nasceu,estado_onde_reside,regiao_onde_reside,regiao_onde_nasceu
20000,20001,1,25,2,37,2,1,5.0,-0.05199,0.0,0.0,"(30, 40]",0.079093,9,10,3,0
20001,20002,1,10,1,31,2,1,1.0,-0.05199,9.0,5.0,"(30, 40]",-0.051922,19,18,3,4
20002,20003,1,10,1,18,2,1,12.0,-0.05199,9.0,2.0,"(10, 20]",-0.054292,23,22,4,5
20003,20004,1,10,1,55,2,1,25.0,-0.05199,9.0,1.0,"(50, 60]",0.040983,15,14,1,1
20004,20005,1,10,1,55,1,1,11.0,-0.05199,0.0,1.0,"(50, 60]",-0.059056,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,24996,1,10,1,49,2,0,8.0,-0.05199,9.0,1.0,"(40, 50]",-0.025709,10,9,1,1
24996,24997,1,5,2,18,2,1,2.0,-0.05199,0.0,0.0,"(10, 20]",-0.031426,18,17,4,5
24997,24998,1,10,1,29,1,1,2.0,-0.05199,9.0,5.0,"(20, 30]",-0.044765,10,10,3,1
24998,24999,1,25,1,31,2,1,10.0,-0.05199,0.0,0.0,"(30, 40]",-0.059056,14,13,2,2


In [61]:
test.drop(columns=[
    'regiao_onde_reside', 'regiao_onde_nasceu', 'estado_onde_reside'],
           inplace=True)

/home/hilsts/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Cleaned train

In [62]:
train

,id_solicitante,produto_solicitado,dia_vencimento,sexo,idade,estado_civil,possui_telefone_residencial,meses_na_residencia,valor_patrimonio_pessoal_norm,profissao,ocupacao,inadimplente,idade_bin,renda_total_norm,estado_onde_nasceu,estado_onde_reside,regiao_onde_reside,regiao_onde_nasceu
0,1,1,10,2,85,2,1,12.0,-0.05199,9.0,1.0,0.0,"(80, 90]",-0.046670,6,5,1,1
1,2,1,25,1,38,1,1,5.0,-0.05199,2.0,5.0,0.0,"(30, 40]",-0.056198,25,24,1,1
2,3,1,20,1,37,2,1,1.0,-0.05199,0.0,0.0,1.0,"(30, 40]",-0.035237,5,4,1,1
3,4,1,20,2,37,1,1,1.0,-0.05199,9.0,2.0,1.0,"(30, 40]",-0.048576,23,22,4,5
4,5,7,1,1,51,1,1,1.0,-0.05199,9.0,5.0,1.0,"(50, 60]",0.030217,5,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1,10,2,27,2,1,0.0,-0.05199,9.0,1.0,0.0,"(20, 30]",-0.052101,11,10,3,4
19996,19997,1,20,1,26,2,1,3.0,-0.05199,9.0,2.0,0.0,"(20, 30]",-0.059056,6,5,1,1
19997,19998,1,10,1,63,2,1,25.0,-0.05199,9.0,1.0,0.0,"(60, 70]",-0.061819,5,4,1,1
19998,19999,1,5,1,84,1,0,30.0,-0.05199,0.0,0.0,0.0,"(80, 90]",-0.056198,15,19,1,1


In [63]:
train.drop(columns=[
    'regiao_onde_reside', 'regiao_onde_nasceu', 'estado_onde_reside'],
           inplace=True)

/home/hilsts/.local/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [64]:
train.groupby('inadimplente').size()

inadimplente
0.0    10000
1.0    10000
dtype: int64

#### Split axes

In [65]:
y_train = train['inadimplente'].to_numpy()
print(set(y_train))
X_train = train.drop(columns=['inadimplente']).select_dtypes(include='number').to_numpy()
X_test = test.select_dtypes(include='number').to_numpy()
print(f'X_test len: {len(X_test)}')


print(X_train.shape)
print(y_train.shape)

# knn = KNeighborsClassifier(n_neighbors=100)

# # # Fit the classifier to the training data
# knn.fit(X_train, y_train)

# pred = knn.predict(X_test)
# print(f'pred len {len(pred)}')
# print(set(pred))
# print(type(pred))

# prediciton_df = pd.DataFrame({'id_solicitante' : test['id_solicitante'].to_numpy(),
#                               'inadimplente' : pred})
# # prediciton_df['inadimplente'] = pd.Series(pred)

# prediciton_df.to_csv(f'data/desafio 1/prediction_knn_100.csv')


{0.0, 1.0}
X_test len: 5000
(20000, 13)
(20000,)


In [9]:
def knn(n):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    pred = knn.predict(X_test)
    prediciton_df = pd.DataFrame({'id_solicitante' : test['id_solicitante'].to_numpy(),
                              'inadimplente' : pred})
    
    prediciton_df.to_csv(f'data/desafio 1/prediction_knn_{n}.csv')

In [131]:
pred = knn.predict(X_test)
print(f'pred len {len(pred)}')
print(set(pred))
print(type(pred))

prediciton_df = pd.DataFrame({'id_solicitante' : test['id_solicitante'].to_numpy(),
                              'inadimplente' : pred})
# prediciton_df['inadimplente'] = pd.Series(pred)

prediciton_df.to_csv(f'data/desafio 1/prediction_knn_{n}.csv')


pred len 5000
{0, 1}
<class 'numpy.ndarray'>


,id_solicitante,inadimplente
0,20001,0
1,20002,0
2,20003,0
3,20004,0
4,20005,0
...,...,...
4995,24996,0
4996,24997,0
4997,24998,0
4998,24999,0


### Logistic Regression

In [66]:
lr = LogisticRegression(penalty='l1', solver='liblinear')
lr.fit(X_train, y_train)
pred_lr = lr.predict(X_test)
prediciton_lr_df = pd.DataFrame({'id_solicitante' : test['id_solicitante'].to_numpy(),
                              'inadimplente' : pred_lr})

In [67]:
prediciton_lr_df['inadimplente'] = prediciton_lr_df['inadimplente'].astype(int)

In [68]:
prediciton_lr_df.groupby('inadimplente').size()

inadimplente
0    2054
1    2946
dtype: int64

In [69]:
prediciton_lr_df.to_csv(f'data/desafio 1/predictions/prediction_lr_6.csv', index=False)

### Decision Tree

In [70]:
dtc = DecisionTreeClassifier(criterion='entropy', max_depth=10)
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)
prediciton_dtc_df = pd.DataFrame({'id_solicitante' : test['id_solicitante'].to_numpy(),
                              'inadimplente' : dtc_pred})

In [71]:
prediciton_dtc_df['inadimplente'] = prediciton_dtc_df['inadimplente'].astype(int)

In [72]:
prediciton_dtc_df.groupby('inadimplente').size()

inadimplente
0    2161
1    2839
dtype: int64

In [73]:
prediciton_dtc_df.to_csv(f'data/desafio 1/predictions/prediction_dtc_6.csv', index=False)

In [44]:
prediciton_df['inadimplente'] = 1
prediciton_df.to_csv('data/desafio 1/predictions/prediction_full_1.csv', index=False)

### SVM